# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [119]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from scipy.sparse import csr_matrix, hstack
import re

In [9]:
spam_data = pd.read_csv(r"D:/Applied-Text-Mining-in-Python/assignments/assignment 3/assets/spam.csv")

In [12]:
spam_data.target.unique()

array(['ham', 'spam'], dtype=object)

In [13]:
spam_data.target[:10]

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: target, dtype: object

In [14]:
# spam -> 1, ham -> 0
spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [22]:
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], spam_data['target'], random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [19]:
(spam_data.target == 1).sum() + (spam_data.target == 0).sum() == spam_data.shape[0]

True

In [20]:
(spam_data.target == 1).sum(), (spam_data.target == 0).sum()

(747, 4825)

In [15]:
def answer_one():
    return (spam_data.target == 1).sum() / spam_data.shape[0] * 100

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [27]:
sorted(CountVectorizer().fit(X_train).get_feature_names_out(), reverse = True, key = lambda token: len(token))[0]

'com1win150ppmx3age16subscription'

In [28]:
def answer_two():
    return sorted(CountVectorizer().fit(X_train).get_feature_names_out(), reverse = True, key = lambda token: len(token))[0]

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [32]:
def answer_three():

    vector = CountVectorizer().fit(X_train)
    train_x_vectorized = vector.transform(X_train)
    model = MultinomialNB(alpha = 0.1)
    model.fit(train_x_vectorized, y_train)
    predictions = model.predict(vector.transform(X_test))
    return roc_auc_score(y_test, predictions)

answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [49]:
# x -> documents
# y -> features

vector = TfidfVectorizer().fit(X_train)
sparse_mat = vector.transform(X_train)

print(f"There are {vector.get_feature_names_out().size} features in the vector")
print(f"There are {sparse_mat.max(axis = 0).toarray().size} columns in the SciPy sparse matrix")

There are 7354 features in the vector
There are 7354 columns in the SciPy sparse matrix


In [55]:
# an array of column maximums

maxes = sparse_mat.max(axis = 0).toarray()[0]
maxes

array([0.23731416, 0.36138463, 0.22862438, ..., 0.27814246, 0.36296157,
       0.3487995 ])

In [68]:
np.arange(1, 101)[-20:]

array([ 81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100])

In [61]:
# smallest
smallest = maxes.argsort()[:20]

# largest
largest = maxes.argsort()[-20:]

In [62]:
vector.get_feature_names_out()[smallest]

array(['sympathetic', 'healer', 'aaniye', 'dependable', 'companion',
       'listener', 'athletic', 'exterminator', 'psychiatrist', 'pest',
       'determined', 'chef', 'courageous', 'stylist', 'psychologist',
       'organizer', 'pudunga', 'venaam', 'diwali', 'mornings'],
      dtype=object)

In [69]:
maxes[smallest]

array([0.07447452, 0.07447452, 0.07447452, 0.07447452, 0.07447452,
       0.07447452, 0.07447452, 0.07447452, 0.07447452, 0.07447452,
       0.07447452, 0.07447452, 0.07447452, 0.07447452, 0.07447452,
       0.07447452, 0.07447452, 0.07447452, 0.09124961, 0.09124961])

In [85]:
s_smallest = pd.Series(maxes[smallest], index = vector.get_feature_names_out()[smallest]).sort_values()
s_smallest

sympathetic     0.074475
venaam          0.074475
pudunga         0.074475
organizer       0.074475
psychologist    0.074475
stylist         0.074475
courageous      0.074475
chef            0.074475
determined      0.074475
pest            0.074475
psychiatrist    0.074475
exterminator    0.074475
athletic        0.074475
listener        0.074475
companion       0.074475
dependable      0.074475
aaniye          0.074475
healer          0.074475
diwali          0.091250
mornings        0.091250
dtype: float64

In [86]:
pd.concat([s_smallest[:18].sort_index(), s_smallest[18:].sort_index()])

aaniye          0.074475
athletic        0.074475
chef            0.074475
companion       0.074475
courageous      0.074475
dependable      0.074475
determined      0.074475
exterminator    0.074475
healer          0.074475
listener        0.074475
organizer       0.074475
pest            0.074475
psychiatrist    0.074475
psychologist    0.074475
pudunga         0.074475
stylist         0.074475
sympathetic     0.074475
venaam          0.074475
diwali          0.091250
mornings        0.091250
dtype: float64

In [63]:
vector.get_feature_names_out()[largest]

array(['blank', 'tick', '645', 'done', 'too', 'anytime', 'beerage',
       'where', 'ok', 'thank', 'yup', 'nite', 'lei', 'anything', 'er',
       'thanx', 'okie', 'home', 'havent', '146tf150p'], dtype=object)

In [70]:
maxes[largest]

array([0.93270158, 0.9801659 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [87]:
s_largest = pd.Series(maxes[largest], index = vector.get_feature_names_out()[largest]).sort_values(ascending = False)
s_largest

yup          1.000000
nite         1.000000
havent       1.000000
home         1.000000
okie         1.000000
thanx        1.000000
er           1.000000
anything     1.000000
lei          1.000000
146tf150p    1.000000
thank        1.000000
ok           1.000000
where        1.000000
beerage      1.000000
anytime      1.000000
too          1.000000
done         1.000000
645          1.000000
tick         0.980166
blank        0.932702
dtype: float64

In [89]:
pd.concat([s_largest[:18].sort_index(), s_largest[18:]])

146tf150p    1.000000
645          1.000000
anything     1.000000
anytime      1.000000
beerage      1.000000
done         1.000000
er           1.000000
havent       1.000000
home         1.000000
lei          1.000000
nite         1.000000
ok           1.000000
okie         1.000000
thank        1.000000
thanx        1.000000
too          1.000000
where        1.000000
yup          1.000000
tick         0.980166
blank        0.932702
dtype: float64

In [93]:
def answer_four():

    vector = TfidfVectorizer().fit(X_train)
    sparse_mat = vector.transform(X_train)
    maxes = sparse_mat.max(axis = 0).toarray()[0]
    smallest = maxes.argsort()[:20]
    largest = maxes.argsort()[-20:]
    s_smallest = pd.Series(maxes[smallest], index = vector.get_feature_names_out()[smallest]).sort_values()
    s_largest = pd.Series(maxes[largest], index = vector.get_feature_names_out()[largest]).sort_values(ascending = False)
    
    _s = pd.concat([s_smallest[:18].sort_index(), s_smallest[18:].sort_index()])
    _l = pd.concat([s_largest[:18].sort_index(), s_largest[18:]])
    return _s, _l
    
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [96]:
def answer_five():
    vector = TfidfVectorizer(min_df = 3).fit(X_train)
    train_x_vectorized = vector.transform(X_train)
    
    model = MultinomialNB(alpha = 0.1)
    model.fit(train_x_vectorized, y_train)
    
    predictions = model.predict(vector.transform(X_test))
    return roc_auc_score(y_test, predictions)
    
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [104]:
spam_data.text.loc[spam_data.target == 1].apply(lambda text: len(text)).mean()

138.8661311914324

In [105]:
spam_data.text.loc[spam_data.target == 0].apply(lambda text: len(text)).mean()

71.02362694300518

In [109]:
def answer_six():

    return (spam_data.text.loc[spam_data.target == 0].apply(lambda text: len(text)).mean(),
                spam_data.text.loc[spam_data.target == 1].apply(lambda text: len(text)).mean())

answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [112]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """ 
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [114]:
X_train.apply(lambda text: len(text))

872      31
831     130
1273     66
3314    146
4929    124
       ... 
4931     37
3264    173
1653    147
2607     62
2732     82
Name: text, Length: 4179, dtype: int64

In [117]:
def answer_seven():

    len_docs_train = X_train.apply(lambda text: len(text))
    len_docs_test = X_test.apply(lambda text: len(text))
    
    vector = TfidfVectorizer(min_df = 5).fit(X_train)
    
    train_x_vectorized = vector.transform(X_train)
    new_train_x_vectorized = add_feature(train_x_vectorized, len_docs_train)
    
    test_x_vectorized = vector.transform(X_test)
    new_test_x_vectorized = add_feature(test_x_vectorized, len_docs_test)
    
    model = SVC(C = 10000)
    model.fit(new_train_x_vectorized, y_train)
    
    predictions = model.decision_function(new_test_x_vectorized)
    return roc_auc_score(y_test, predictions)
    
answer_seven()

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [120]:
re.findall("\d", "6jg346t763fhuq3i6r")

['6', '3', '4', '6', '7', '6', '3', '3', '6']

In [121]:
def answer_eight():
    import re
    return (spam_data.text.loc[spam_data.target == 0].apply(lambda text: len(re.findall("\d", text))).mean(),
            spam_data.text.loc[spam_data.target == 1].apply(lambda text: len(re.findall("\d", text))).mean())

answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.linear_model import LogisticRegression

def answer_nine():

    # YOUR CODE HERE
    raise NotImplementedError()
    return #Your answer here

answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [ ]:
def answer_ten():
    import re
    return (spam_data.text.loc[spam_data.target == 0].apply(lambda text: len(re.findall("\W", text))).mean(),
            spam_data.text.loc[spam_data.target == 1].apply(lambda text: len(re.findall("\W", text))).mean())

answer_ten()

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [ ]:
def answer_eleven():

    # YOUR CODE HERE
    raise NotImplementedError()
    return #Your answer here

answer_eleven()